In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib

## Townhall data

In [2]:
# each page has a summary table that rolls up results at the state level
# get rid of it
def cond(x):
    if x:
        return x.startswith("table ec-table") and not "table ec-table ec-table-summary" in x
    else:
        return False

In [3]:
# list of state abbreviations
states = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

# headers for csv export
data = [['state_abbr', 'county_name', 'party', 'votes_total']]

In [4]:
# loop through each state's web page http://townhall.com/election/2016/president/%s/county, where %s is the state abbr
for state in states:
    r = urllib.urlopen('http://townhall.com/election/2016/president/' + state + '/county').read()
    soup = BeautifulSoup(r, "html.parser")

    # loop through each <table> tag with .ec-table class
    tables = soup.findAll('table', attrs={'class':cond})

    for table in tables:
        if table.findParent("table") is None:
            table_body = table.find('tbody')

            rows = table_body.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                # first tbody tr has four td
                if len(cols) == 4:
                    # strip text from each td
                    divs = cols[0].find_all('div')
                    county = divs[0].text.strip()
                    party = cols[1]['class'][0]
                    total_votes = int(cols[2].text.strip().replace(',','').replace('-','0'))
                # all other tbody tr have three td
                else:
                    party = cols[1]['class'][0]
                    total_votes = int(cols[1].text.strip().replace(',','').replace('-','0'))
                    
                #combine each row's results
                rowData = [state,county,party,total_votes]
                data.append(rowData)

In [5]:
townhall = pd.DataFrame(data) # throw results in dataframe
new_header = townhall.iloc[0] #grab the first row for the header
townhall = townhall[1:] #take the data less the header row
townhall.columns = new_header #set the header row as the df header
townhall['votes_total'] = townhall['votes_total'].astype('float64')
print(townhall.shape[0])
townhall[townhall['state_abbr'] == 'MD']

14184


,state_abbr,county_name,party,votes_total
5269,MD,Allegany,GOP,20025.0
5270,MD,Allegany,DEM,6665.0
5271,MD,Allegany,LIB,778.0
5272,MD,Allegany,GRN,336.0
5273,MD,Anne Arundel,DEM,116074.0
5274,MD,Anne Arundel,GOP,114509.0
5275,MD,Anne Arundel,LIB,9365.0
5276,MD,Anne Arundel,GRN,2991.0
5277,MD,Baltimore City,DEM,178562.0
5278,MD,Baltimore City,GOP,22726.0


In [6]:
# strip out state abbreviations and county names from townhall
townhall_counties = townhall[['state_abbr','county_name']].drop_duplicates().reset_index().drop('index',1)
print('Townhall data has ' + str(townhall_counties.shape[0]) + ' counties')
townhall_counties[townhall_counties['state_abbr'] == 'MD']

Townhall data has 3112 counties


,state_abbr,county_name
1163,MD,Allegany
1164,MD,Anne Arundel
1165,MD,Baltimore City
1166,MD,Baltimore County
1167,MD,Calvert
1168,MD,Caroline
1169,MD,Carroll
1170,MD,Cecil
1171,MD,Charles
1172,MD,Dorchester


In [7]:
# combine state and county names
townhall['combined'] = townhall['state_abbr'] + townhall['county_name'].apply(lambda x: x.replace('County','').replace(' ','').lower())


townhall[townhall['state_abbr'] == 'MD']

,state_abbr,county_name,party,votes_total,combined
5269,MD,Allegany,GOP,20025.0,MDallegany
5270,MD,Allegany,DEM,6665.0,MDallegany
5271,MD,Allegany,LIB,778.0,MDallegany
5272,MD,Allegany,GRN,336.0,MDallegany
5273,MD,Anne Arundel,DEM,116074.0,MDannearundel
5274,MD,Anne Arundel,GOP,114509.0,MDannearundel
5275,MD,Anne Arundel,LIB,9365.0,MDannearundel
5276,MD,Anne Arundel,GRN,2991.0,MDannearundel
5277,MD,Baltimore City,DEM,178562.0,MDbaltimorecity
5278,MD,Baltimore City,GOP,22726.0,MDbaltimorecity


In [8]:
# return sum of votes by state and county
townhall['total_votes'] = townhall['votes_total'].groupby(townhall['combined']).transform('sum')
townhall_counties = townhall.drop('votes_total',axis=1)

# view dataset by selected state
townhall_counties[townhall_counties['state_abbr'] == 'MD']

,state_abbr,county_name,party,combined,total_votes
5269,MD,Allegany,GOP,MDallegany,27804.0
5270,MD,Allegany,DEM,MDallegany,27804.0
5271,MD,Allegany,LIB,MDallegany,27804.0
5272,MD,Allegany,GRN,MDallegany,27804.0
5273,MD,Anne Arundel,DEM,MDannearundel,242939.0
5274,MD,Anne Arundel,GOP,MDannearundel,242939.0
5275,MD,Anne Arundel,LIB,MDannearundel,242939.0
5276,MD,Anne Arundel,GRN,MDannearundel,242939.0
5277,MD,Baltimore City,DEM,MDbaltimorecity,208980.0
5278,MD,Baltimore City,GOP,MDbaltimorecity,208980.0


## Census data

In [9]:
# county_fips data from https://www.census.gov/geo/reference/codes/cou.html
census = pd.read_csv('http://www2.census.gov/geo/docs/reference/codes/files/national_county.txt',sep=',',header=None, dtype=str)
census.columns = ['state_abbr', 'state_fips', 'county_fips', 'county_name', 'fips_class_code']
print(census.shape)
census.head()

(3235, 5)


,state_abbr,state_fips,county_fips,county_name,fips_class_code
0,AL,01,001,Autauga County,H1
1,AL,01,003,Baldwin County,H1
2,AL,01,005,Barbour County,H1
3,AL,01,007,Bibb County,H1
4,AL,01,009,Blount County,H1


In [10]:
census[census['state_abbr'] == 'MD']

,state_abbr,state_fips,county_fips,county_name,fips_class_code
1193,MD,24,001,Allegany County,H1
1194,MD,24,003,Anne Arundel County,H1
1195,MD,24,005,Baltimore County,H1
1196,MD,24,009,Calvert County,H1
1197,MD,24,011,Caroline County,H1
1198,MD,24,013,Carroll County,H1
1199,MD,24,015,Cecil County,H1
1200,MD,24,017,Charles County,H1
1201,MD,24,019,Dorchester County,H1
1202,MD,24,021,Frederick County,H1


In [11]:
# get state and county fips
#census.columns[[0,1,2,3,4]]
fips_codes_census = census.drop(census.columns[[4]],axis=1)
print(fips_codes_census['county_fips'].count())
fips_codes_census.head()

3235


,state_abbr,state_fips,county_fips,county_name
0,AL,01,001,Autauga County
1,AL,01,003,Baldwin County
2,AL,01,005,Barbour County
3,AL,01,007,Bibb County
4,AL,01,009,Blount County


In [12]:
# strip out state abbreviations and state names from census
census_states = census[['state_abbr','state_fips']].drop_duplicates().reset_index().drop('index',1)
# drop US territories
census_states = census_states[(census_states['state_abbr'] != 'AS') & (census_states['state_abbr'] != 'GU') & (census_states['state_abbr'] != 'MP') & (census_states['state_abbr'] != 'PR') & (census_states['state_abbr'] != 'UM') & (census_states['state_abbr'] != 'VI')]
print(str(census_states.shape[0]) + ' states')

51 states


In [13]:
# strip out state abbreviations and county names from census
census_counties = census[['state_abbr','state_fips','county_name','county_fips']].drop_duplicates().reset_index().drop('index',1)
census_counties = census_counties[(census_counties['state_abbr'] != 'AS') & (census_counties['state_abbr'] != 'GU') & (census_counties['state_abbr'] != 'MP') & (census_counties['state_abbr'] != 'PR') & (census_counties['state_abbr'] != 'UM') & (census_counties['state_abbr'] != 'VI')]
print('Census data has ' + str(census_counties.shape[0]) + ' counties')
census_counties[census_counties['state_abbr'] == 'MD']

Census data has 3143 counties


,state_abbr,state_fips,county_name,county_fips
1193,MD,24,Allegany County,001
1194,MD,24,Anne Arundel County,003
1195,MD,24,Baltimore County,005
1196,MD,24,Calvert County,009
1197,MD,24,Caroline County,011
1198,MD,24,Carroll County,013
1199,MD,24,Cecil County,015
1200,MD,24,Charles County,017
1201,MD,24,Dorchester County,019
1202,MD,24,Frederick County,021


In [92]:
census['combined'] = census['state_abbr'] + census['county_name'].apply(lambda x: x.replace('County','').replace(' ','').lower())
census[census['state_abbr'] == 'MD']

,state_abbr,state_fips,county_fips,county_name,fips_class_code,combined
1193,MD,24,001,Allegany County,H1,MDallegany
1194,MD,24,003,Anne Arundel County,H1,MDannearundel
1195,MD,24,005,Baltimore County,H1,MDbaltimore
1196,MD,24,009,Calvert County,H1,MDcalvert
1197,MD,24,011,Caroline County,H1,MDcaroline
1198,MD,24,013,Carroll County,H1,MDcarroll
1199,MD,24,015,Cecil County,H1,MDcecil
1200,MD,24,017,Charles County,H1,MDcharles
1201,MD,24,019,Dorchester County,H1,MDdorchester
1202,MD,24,021,Frederick County,H1,MDfrederick


In [25]:
# join census and townhall data on the 'combined' column
right = townhall.set_index('combined')
left = census.set_index('combined')

combined = left.join(right, lsuffix='', rsuffix='_r')
combined = combined.reset_index()
print('Joined dataset has ' + str(combined.shape[0]) + ' items')

# view data by selected state
combined[combined['state_abbr'] == 'AL']

Joined dataset has 13816 items


,combined,state_abbr,state_fips,county_fips,county_name,fips_class_code,state_abbr_r,county_name_r,party,votes_total,total_votes
29,ALautauga,AL,01,001,Autauga County,H1,AL,Autauga,GOP,18110.0,24661.0
30,ALautauga,AL,01,001,Autauga County,H1,AL,Autauga,DEM,5908.0,24661.0
31,ALautauga,AL,01,001,Autauga County,H1,AL,Autauga,IND,538.0,24661.0
32,ALautauga,AL,01,001,Autauga County,H1,AL,Autauga,IND,105.0,24661.0
33,ALbaldwin,AL,01,003,Baldwin County,H1,AL,Baldwin,GOP,72780.0,94090.0
34,ALbaldwin,AL,01,003,Baldwin County,H1,AL,Baldwin,DEM,18409.0,94090.0
35,ALbaldwin,AL,01,003,Baldwin County,H1,AL,Baldwin,IND,2448.0,94090.0
36,ALbaldwin,AL,01,003,Baldwin County,H1,AL,Baldwin,IND,453.0,94090.0
37,ALbarbour,AL,01,005,Barbour County,H1,AL,Barbour,GOP,5431.0,10390.0
38,ALbarbour,AL,01,005,Barbour County,H1,AL,Barbour,DEM,4848.0,10390.0


In [80]:
# return unique dataset
county_level_combined = combined.drop_duplicates()
print('Combined dataset has ' + str(county_level_combined.shape[0]) + ' total items')
                                                   
# return only D and R results
county_level_combined = county_level_combined[(county_level_combined['party'] == 'GOP') | (county_level_combined['party'] == 'DEM')]
print('Filtered dataset has ' + str(county_level_filtered.shape[0]) + ' D and R items')

# flatten dataset by adding votes by R and D columns
county_level_combined['votes_dem'] = county_level_combined['votes_total'].where(county_level_combined['party'] == 'DEM',0).astype('float32')
county_level_combined['votes_gop'] = county_level_combined['votes_total'].where(county_level_combined['party'] == 'GOP',0).astype('float32')

# drop party and party-level totals and other columns
county_level_combined.drop(['party','votes_total','fips_class_code','state_abbr_r','county_name_r'], axis=1, inplace=True)
# total_results = county_level_combined.drop(['party','votes_total','fips_class_code','state_abbr_r','county_name_r', 'votes_dem', 'votes_gop'], axis=1, inplace=True)
county_level_combined[county_level_combined['state_abbr'] == 'AL']

Combined dataset has 13816 total items
Filtered dataset has 6010 D and R items


,combined,state_abbr,state_fips,county_fips,county_name,total_votes,votes_dem,votes_gop
29,ALautauga,AL,01,001,Autauga County,24661.0,0.0,18110.0
30,ALautauga,AL,01,001,Autauga County,24661.0,5908.0,0.0
33,ALbaldwin,AL,01,003,Baldwin County,94090.0,0.0,72780.0
34,ALbaldwin,AL,01,003,Baldwin County,94090.0,18409.0,0.0
37,ALbarbour,AL,01,005,Barbour County,10390.0,0.0,5431.0
38,ALbarbour,AL,01,005,Barbour County,10390.0,4848.0,0.0
41,ALbibb,AL,01,007,Bibb County,8748.0,0.0,6733.0
42,ALbibb,AL,01,007,Bibb County,8748.0,1874.0,0.0
45,ALblount,AL,01,009,Blount County,25384.0,0.0,22808.0
46,ALblount,AL,01,009,Blount County,25384.0,2150.0,0.0


In [99]:
# pivot data to consolidate
party_pivot = pd.pivot_table(county_level_combined,index=["combined"],values=["votes_dem","votes_gop"],aggfunc=np.sum)
total_pivot = pd.pivot_table(county_level_combined,index=["combined"],values=["total_votes"],aggfunc=np.mean)

# join party and total pivots
combined_pivot = party_pivot.join(total_pivot, lsuffix='', rsuffix='_r')
print('Joined dataset has ' + str(combined_pivot.shape[0]) + ' items')
combined_pivot

# add percentages for each R and D of total votes
# calculate percentage of total vote per major candidates
combined_pivot['per_dem'] = combined_pivot['votes_dem'] / combined_pivot['total_votes']
combined_pivot['per_gop'] = combined_pivot['votes_gop'] / combined_pivot['total_votes']
combined_pivot

# join pivotted and unpivotted data
right = census.set_index('combined')
# left = combined_pivot.set_index('combined')

county_level_final = combined_pivot.join(right, lsuffix='', rsuffix='_r')
county_level_final = county_level_final.reset_index()
print('Joined dataset has ' + str(county_level_final.shape[0]) + ' items')
county_level_final

# create FIPS columns for visualizations
county_level_final['combined_fips'] = county_level_final['state_fips'].apply(lambda x: x.lstrip('0')) + county_level_final['county_fips']
county_level_final

# drop irrelevant columns
county_level_final = county_level_final.drop(['combined','county_fips','state_fips','fips_class_code'], axis=1)
print('Final dataset has ' + str(county_level_final.shape[0]) + ' items')
county_level_final

# view data by selected state
county_level_final[county_level_final['state_abbr'] == 'MD']

Joined dataset has 3005 items
Joined dataset has 3005 items
Final dataset has 3005 items


,votes_dem,votes_gop,total_votes,per_dem,per_gop,state_abbr,county_name,combined_fips
1096,6665.0,20025.0,27804.0,0.239714,0.720220,MD,Allegany County,24001
1097,116074.0,114509.0,242939.0,0.477791,0.471349,MD,Anne Arundel County,24003
1098,189437.0,131009.0,335017.0,0.565455,0.391052,MD,Baltimore County,24005
1099,178562.0,22726.0,208980.0,0.854445,0.108747,MD,Baltimore city,24510
1100,16669.0,24680.0,43445.0,0.383681,0.568075,MD,Calvert County,24009
1101,3747.0,8999.0,13282.0,0.282111,0.677534,MD,Caroline County,24011
1102,24314.0,55593.0,84879.0,0.286455,0.654968,MD,Carroll County,24013
1103,12651.0,27579.0,42309.0,0.299014,0.651847,MD,Cecil County,24015
1104,46063.0,24163.0,72480.0,0.635527,0.333375,MD,Charles County,24017
1105,5695.0,7938.0,14034.0,0.405800,0.565626,MD,Dorchester County,24019


In [100]:
county_level_final.to_csv('2016_US_County_Level_Presidential_Results.csv',sep=',')